### This example illustrates how to use mlflow to track parameters, metrics, artifacts and models while training a simple MNIST model.

In [50]:
import mlflow.fastai
from fastai.vision.all import (
    CategoryBlock,
    DataBlock,
    GrandparentSplitter,
    ImageBlock,
    PILImage,
    URLs,
)
from fastai.vision.all import cnn_learner, get_image_files, parent_label, resnet18, untar_data
import numpy as np

#### Splitting data between training and validation

In [51]:
splitter = GrandparentSplitter(train_name="training", valid_name="testing")

# Prepare DataBlock which is a generic container to quickly build Datasets and DataLoaders
mnist = DataBlock(
    blocks=(ImageBlock(PILImage), CategoryBlock),
    get_items=get_image_files,
    splitter=splitter,
    get_y=parent_label,
)

#### Downloading MNIST data set and creating DataLoader from Datablock

In [55]:
data_path = untar_data(URLs.MNIST)
image_files = get_image_files(data_path)
print('Data located at:', data_path)

data = mnist.dataloaders(data_path, bs=256, num_workers=0)

data.train.show_batch()


Data located at: /home/azureuser/.fastai/data/mnist_png


In [61]:
data.valid_ds[0]

(PILImage mode=RGB size=28x28, TensorCategory(3))

### Creating signature
#### This helps to define what input does our model take. With this info we can easily share model without users having to ponder on what input does a model take

We will register our model with a given signature.This means that our inputs will be tensors of shape (IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS). Since the model expects batches of images, the right input shape is (BATCH_SIZE, IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS).

**input_schema**: Our model can handle variable image sizes.Then, we can indicate a variable input size using -1 in height and width. The same applies to batch size, since we can handle variable sizes.

**output_schema**: Our model will return class probabilities. Since we have 10 classes, then the out of the model has shape (BATCH_SIZE, NUM_CLASSES) or (-1, 10).

In [62]:
import numpy as np
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, TensorSpec

input_schema = Schema([
  TensorSpec(np.dtype(np.uint8), (-1, -1, -1, 3)),
])
output_schema = Schema([
  TensorSpec(np.dtype(np.float32), (-1, 10)),
])
signature = ModelSignature(inputs=input_schema, outputs=output_schema)

##### Enabling autologging from mlflow

In [ ]:
# mlflow.fastai.autolog()

##### Create Learner model
Once our dataset is ready, it's time to create our NN. CNN represents a very convenient way to solve Computer Vision problems, specially when combined with transfer learning. We use transfer learning with a pretrained image classification models

In [63]:

learn = cnn_learner(data, resnet18)

##### Using different loading function

In [64]:
%%writefile fastai_model_loader.py
import os
import numpy as np
import pandas as pd

class _FastaiModelWrapperPatch:
    def __init__(self, learner):
        self.learner = learner

    def predict(self, dataframe):
        dl = self.learner.dls.test_dl(dataframe)
        preds, _ = self.learner.get_preds(dl=dl)
        return pd.Series(map(np.array, preds.numpy())).to_frame('predictions')

def _load_model(path):
    from fastai.learner import load_learner

    return load_learner(os.path.abspath(path))

def _load_pyfunc(path):
    """
    Load PyFunc implementation. Called by ``pyfunc.load_pyfunc``.
    :param path: Local filesystem path to the MLflow Model with the ``fastai`` flavor.
    """
    print("Model loaded from:",path)
    return _FastaiModelWrapperPatch(_load_model(path))

Writing fastai_model_loader.py


##### Training model

In [65]:
saved_model = "/home/azureuser/.fastai/model.fastai"

with mlflow.start_run() as run:
    # Train and fit with default or supplied command line arguments
    learn.fit_one_cycle(1, 0.1)
    learn.export(saved_model)
    model = mlflow.pyfunc.log_model("model", 
                                    registered_model_name="digits_cnn_model",
                                    data_path=saved_model, 
                                    code_path=["./fastai_model_loader.py"], 
                                    loader_module="fastai_model_loader", 
                                    signature=signature)

epoch,train_loss,valid_loss,time
0,0.103479,0.044797,01:41


Registered model 'digits_cnn_model' already exists. Creating a new version of this model...
2022/02/25 20:24:44 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: digits_cnn_model, version 2
Created version '2' of model 'digits_cnn_model'.


#### Loading trained model from the run

In [66]:
model_uri = "runs:/{}/model".format(run.info.run_id)
loaded_model = mlflow.pyfunc.load_model(model_uri)
model_uri

Model loaded from: /tmp/tmpmcpel_3t/model/data/model.fastai


'runs:/2b300ab1-93de-46d4-938a-15f411c77576/model'

 #### Creating a sample batch for predcition to a rest endpoint

In [67]:
from fastai.vision.core import load_image

# sample_img = load_image(image_files[1])
sample_img_arr = np.array(data.train_ds[0][0])

sample_batch = sample_img_arr.reshape(1, sample_img.height, sample_img.width,3)

In [68]:
data.train_ds[0][0]

##### Local prediction

In [69]:
loaded_model.predict(sample_batch)

,predictions
0,"[1.0928232e-08, 7.922903e-08, 1.7171018e-05, 0.9996363, 2.0671924e-07, 5.190059e-06, 8.323384e-07, 1.8514454e-07, 0.00010446645, 0.00023542823]"


### Running the model as a local endpoint

In [70]:
import json
import requests

inference_request = {
    "instances": sample_batch.tolist()
}

In [71]:
endpoint = "http://localhost:5000/invocations"
response = requests.post(endpoint, json=inference_request)

response.json()

[{'predictions': [1.0928231652940212e-08,
   7.922903222379318e-08,
   1.7171018043882214e-05,
   0.9996362924575806,
   2.0671923550708016e-07,
   5.190058800508268e-06,
   8.323384008690482e-07,
   1.8514454325213592e-07,
   0.00010446645319461823,
   0.00023542823328170925]}]